In [1]:
import numpy  as np
import pandas as pd
from metrics import MAE
from tqdm import tqdm

In [2]:
X_train = pd.read_pickle("../../../Datasets/final/X_train.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
c = X_train.columns.tolist()
X_val = pd.read_pickle("../../../Datasets/final/X_val.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
X_test = pd.read_pickle("../../../Datasets/final/X_test.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
y_train = pd.read_pickle("../../../Datasets/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../../../Datasets/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../../../Datasets/final/y_test.pkl").to_numpy()

In [3]:
X_train.insert(X_train.shape[1], "rating", y_train.T[0])

In [4]:
state_columns = [i for i in c if "state" in i]

In [5]:
mean_per_state_dict = {}
for state in state_columns:
    state_df = X_train.groupby(state).agg({"rating": "mean"}).reset_index()
    mean_per_state_dict[state] = state_df[state_df[state] == 1]["rating"].item()

In [6]:
mean_per_state_dict

{'state_AL': 3.7559808612440193,
 'state_AR': 3.7404702122803015,
 'state_AZ': 3.7855842280404346,
 'state_CA': 3.8718092468333363,
 'state_CO': 3.8026193668144788,
 'state_CT': 3.802923518663534,
 'state_DE': 3.823994726433751,
 'state_GA': 3.775103489059728,
 'state_IA': 3.762487852283771,
 'state_ID': 3.8983527521092807,
 'state_IL': 3.8477072578196174,
 'state_IN': 4.017468074192068,
 'state_KS': 3.7370590929912963,
 'state_KY': 3.7780190614066838,
 'state_LA': 3.576687116564417,
 'state_MA': 3.946958324397741,
 'state_MD': 3.767489199069458,
 'state_ME': 3.81025641025641,
 'state_MI': 3.7669621273166802,
 'state_MN': 3.767685851318945,
 'state_MO': 3.8017885220125787,
 'state_MS': 3.5296442687747036,
 'state_MT': 3.858611825192802,
 'state_NC': 3.819687748081503,
 'state_ND': 3.4686468646864688,
 'state_NE': 3.54320987654321,
 'state_NH': 3.981673306772908,
 'state_NJ': 3.909345674251189,
 'state_NM': 3.7369634111207146,
 'state_NV': 3.7892659826361483,
 'state_NY': 3.754841873008

In [7]:
y_pred_val = []
for i in tqdm(range(len(X_val))):
    for j in range(len(X_val.iloc[i][state_columns])):
        if X_val.iloc[i][state_columns][j] == 1:
            y_pred_val.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [08:59<00:00, 299.21it/s]


In [8]:
MAE(predictions=y_pred_val, labels=y_val)[0]

1.0727434910882276

In [9]:
y_pred = []
for i in tqdm(range(len(X_test))):
    for j in range(len(X_test.iloc[i][state_columns])):
        if X_test.iloc[i][state_columns][j] == 1:
            y_pred.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [09:00<00:00, 298.74it/s]


In [10]:
MAE(predictions=y_pred, labels=y_test)[0]

1.0676916003156327